In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
activity = pd.read_csv("user_vector.csv").drop("Unnamed: 0", axis=1)
label = pd.read_csv("data/train_label.csv").drop("Unnamed: 0", axis=1)

In [ ]:
activity.head()

In [ ]:
label = label.sort_values("acc_id")
label.head()

<br></br><br></br><br></br>

In [ ]:
id_lst = [int(x) for x in label.acc_id.tolist()]
print(len(id_lst))

In [ ]:
activity_dic = {}
for user in id_lst :
    activity_dic[user] = []

In [ ]:
print(len(activity.head().values[0]))
activity.head().values[0]

In [ ]:
for data in activity.values :
    activity_dic[int(data[0])].append([int(data[1])]+list(data[2:]))

In [ ]:
activity_lst = [list(y) for y in activity_dic.items()]
activity_lst[0]

In [ ]:
day_1_lst = [x for x in activity_lst]
print(len(day_1_lst))

In [ ]:
day_1_dic = {}
day_1_id_lst = [x[0] for x in day_1_lst]

In [ ]:
START_TOKEN = [1] + [0]*52
EMPTY_TOKEN = [0,1] + [0]*51
END_TOKEN = [0,0,1] + [0]*50

In [ ]:
for user in day_1_id_lst :
    day_1_dic[(user, 0)] = START_TOKEN
    day_1_dic[(user, 1)] = EMPTY_TOKEN
    day_1_dic[(user, 2)] = EMPTY_TOKEN
    day_1_dic[(user, 3)] = EMPTY_TOKEN
    day_1_dic[(user, 4)] = EMPTY_TOKEN
    day_1_dic[(user, 5)] = EMPTY_TOKEN
    day_1_dic[(user, 6)] = EMPTY_TOKEN
    day_1_dic[(user, 7)] = EMPTY_TOKEN
    day_1_dic[(user, 8)] = EMPTY_TOKEN
    day_1_dic[(user, 9)] = END_TOKEN


In [ ]:
for data in day_1_lst :
    user = data[0]
    lst = data[1]
    
    for idx, data2 in enumerate(lst) :
        if idx == 0 :
            first_week = data2[0]
            
            for idx2 in range(first_week) :
                day_1_dic[(user, idx2)] = START_TOKEN
                
        week = data2[0]
        day_1_dic[(user, week)] = data2[1:]

In [ ]:
label_dic = {"week":0 , "month" :1, "2month":2, "retained":3}

In [ ]:
label2 = label.sort_values(by="acc_id")
label2["label"] = label2["label"].map(lambda x : label_dic[x])

label2.head()

<br></br><br></br><br></br>

In [ ]:
def one_hot(lst, num_class=4) :
    return np.eye(num_class)[lst]

In [ ]:
day_1_total_lst = []
temp1 = list(day_1_dic.values())

last1=0

In [ ]:
for now in range(0,len(temp1)+1,10) :
    if now == 0 :
        last1 = now
        continue
    
    day_1_total_lst.append(temp1[last1:now])
    last1=now

In [ ]:
print(len(day_1_total_lst))

In [ ]:
label_dic = label2.set_index("acc_id").to_dict()['label']
print(len(label_dic))

In [ ]:
day_1_total_label = one_hot([label_dic[x] for x in day_1_id_lst])

In [ ]:
print(len(day_1_total_label))

In [ ]:
idx1 = len(day_1_total_lst)//5 *4

day_1_training_lst = np.array(day_1_total_lst[:idx1])
day_1_valid_lst = np.array(day_1_total_lst[idx1:])

day_1_training_label = np.array(day_1_total_label[:idx1])
day_1_valid_label = np.array(day_1_total_label[idx1:])

<br></br><br></br><br></br>

# Model
- Transformer with denseNet
    - Encoding된 vector를 stack하는 방식
    - 마지막에 1x1 CNN
    
    
- need to customize
    - QK_T_dk -> QK_T
    - sinusoid PE -> other PE
    - pooling
    
    
- https://github.com/jadore801120/attention-is-all-you-need-pytorch/blob/4f4a192f0fd272102c8852b00b1007dffd292b90/transformer/Models.py#L11
- https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
- http://nlp.seas.harvard.edu/2018/04/03/attention.html


In [ ]:
class Transformer_Classifer() :
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
    def embedding(self, X_input, emb_dim, emb_activation) :
        if emb_dim == X_input.get_shape().as_list()[2] :
            emb_vector = X_input
            
        else :
            emb_vector = tf.layers.dense(X_input, emb_dim, activation=emb_activation)
            
        return emb_vector
    
    def positional_encoding(self, X_input) :
        batch, position, dim = X_input.get_shape().as_list()
        position_enc = np.array([list([[pos / np.power(10000, 2*i/dim) for i in range(dim)] 
                                if pos != 0 else np.zeros(dim) for pos in range(position)])]*batch)

        position_enc[:, 1:, 0::2] = np.sin(position_enc[:, 1:, 0::2])
        position_enc[:, 1:, 1::2] = np.cos(position_enc[:, 1:, 1::2])

        pos_enc_vector = tf.constant(position_enc, dtype=tf.float32, shape=[batch, position, dim])
        return pos_enc_vector
        
    def scaled_dot_product_attention(self, Q, K, V, dk) :
        QKT_dk = tf.matmul(Q,K, transpose_b=True) / tf.sqrt(dk)
        attention = tf.nn.softmax(QKT_dk)
        attended_vector = tf.matmul(attention,V)
        
        return attended_vector
        
    def multihead_attention(self, Q, K, V, h) :
        batch, position, dim_q = Q.get_shape().as_list()
        batch, position, dim_k = K.get_shape().as_list()
        batch, position, dim_v = V.get_shape().as_list()
        dq = dim_q//h
        dk = dim_k//h
        dv = dim_v//h
        
        W_Q = tf.Variable(tf.random_normal([h, batch, dim_q, dq]))
        W_K = tf.Variable(tf.random_normal([h, batch, dim_k, dk]))
        W_V = tf.Variable(tf.random_normal([h, batch, dim_v, dv]))
        W_O = tf.Variable(tf.random_normal([batch, dk*h, dim_k]))
    
        head_lst = []
        for idx in range(h) :
            head_lst.append(self.scaled_dot_product_attention(tf.matmul(Q, W_Q[idx]), 
                                                              tf.matmul(K, W_K[idx]), 
                                                              tf.matmul(V, W_V[idx]), 
                                                              tf.constant(dk, tf.float32)))
        
        multihead_attention = tf.concat(head_lst, axis=2)
        linear_projection= tf.matmul(multihead_attention, W_O)
        
        return linear_projection
        
    def normarlization(self, X_input) :
        return tf.contrib.layers.layer_norm(X_input)
    
    def feedforward_network(self, X_input, ffn_dim) :
        batch, position, dim =  X_input.get_shape().as_list()
        
        ff_vector1 = tf.layers.dense(X_input, ffn_dim, activation=tf.nn.leaky_relu)
        dropout = tf.layers.dropout(ff_vector1, training=self.training)
        ff_vector2 = tf.layers.dense(dropout, dim)
        
        return ff_vector2
    
    def resnet(self, X_input1, X_input2) :
        return X_input1 + X_input2
        
    def encoder_layer(self, X_input, h, ffn_dim) :
        attended_vector = self.multihead_attention(X_input, X_input, X_input, h)
        sublayer1 = self.resnet(X_input, attended_vector)
        norm1 = self.normarlization(sublayer1)
        
        ffn_vector = self.feedforward_network(norm1, ffn_dim)
        sublayer2 = self.resnet(norm1, ffn_vector)
        norm2 = self.normarlization(sublayer2)
        
        return norm2
    
    def classifier(self, X_input, fc_dim, fc_activation, output_class) : 
        layer1 = tf.layers.dense(inputs=X_input, units=fc_dim, activation=fc_activation)
        dropout1 = tf.layers.dropout(layer1, training=self.training)
        layer2 = tf.layers.dense(inputs=dropout1, units=fc_dim//10)

        batch, position, dim =  layer2.get_shape().as_list()
        dimension = position*dim
        flat = tf.reshape(layer2, [batch, dimension])

        layer3 = tf.layers.dense(inputs=flat, units=dimension*2, activation=fc_activation)
        dropout2 = tf.layers.dropout(layer3, training=self.training)
        layer4 = tf.layers.dense(inputs=dropout2, units=output_class)

        return layer4
  
    def build(self, batch_size, input_length, input_dim, N, emb_dim, emb_activation, h, ffn_dim, fc_dim, fc_activation, output_class) :
        with tf.variable_scope(self.name) :
            
            ## Setting ##
            # input  : ? x input_length x input_dim
            self.X = tf.placeholder(tf.float32, [batch_size, input_length, input_dim])
            self.Y = tf.placeholder(tf.float32, [batch_size, output_class])
            self.learning_rate =  tf.placeholder(tf.float32)
            self.training = tf.placeholder(tf.bool)
            #############
            
            
            ## Encoder ##
            # input  : ? x input_length x input_dim
            # output : ? x input_length x emb_dim
            embedded_vector = self.embedding(self.X, emb_dim, emb_activation)
            pos_encoded_vector = embedded_vector + self.positional_encoding(embedded_vector)
            encoder_input = pos_encoded_vector
            
            for idx in range(N) :
                encoder_input = self.encoder_layer(encoder_input, h, ffn_dim)
            self.encoder_output = encoder_input
            #############
            
            
            ## Classifier ##
            # input  : ? x input_length x emb_dim
            # output : ? x input_length x fc_dim
            self.logit = self.classifier(self.encoder_output, fc_dim, fc_activation, output_class)
            self.softmax_logit = tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y)
            ################
            
            
            ## Learning ##
            self.cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logit, labels=self.Y))

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope=self.name)
            with tf.control_dependencies(update_ops):
                self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
            
            correct_prediction = tf.equal(tf.argmax(self.logit, 1), tf.argmax(self.Y, 1))     
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    
            ##############
            
    def predict(self, X_input, batch_size, training=False):
        N = X_input.shape[0]
        total_logit = []
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            feed_dict = {self.X: X_batch, self.training: False}
                
            step_logit = self.sess.run(self.logit, feed_dict=feed_dict)
            total_logit.append(step_logit)
            
        return np.concatenate(total_logit, axis=0)

    def get_accuracy(self, X_input, Y_input, batch_size, training=False):
        N = X_input.shape[0]
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            step_acc = self.sess.run(self.accuracy, feed_dict=feed_dict)
            total_acc += step_acc * X_batch.shape[0]
            
        total_acc /= N
        return total_acc

    def train(self, X_input, Y_input, learning_rate, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: X_input, self.Y: Y_input, self.learning_rate: learning_rate, self.training: training})
    
    def evaluate(self, X_input, Y_input, batch_size):
        N = X_input.shape[0]
            
        total_loss = 0
        total_acc = 0
            
        for i in range(0, N, batch_size):
            X_batch = X_input[i:i + batch_size]
            Y_batch = Y_input[i:i + batch_size]
            feed_dict = {self.X: X_batch, self.Y: Y_batch, self.training: False}
                
            loss = self.cost
            accuracy = self.accuracy
                
            step_loss, step_acc = self.sess.run([loss, accuracy], feed_dict=feed_dict)
                
            total_loss += step_loss * X_batch.shape[0]
            total_acc += step_acc * X_batch.shape[0]
            
        total_loss /= N
        total_acc /= N
            
        return total_loss, total_acc

In [ ]:
tf.reset_default_graph() 

<br></br><br></br><br></br>

In [ ]:
learning_rate1 = 0.05
learning_rate2 = 0.02
learning_rate3 = 0.01
learning_rate4 = 0.005

total_epoch = 80
batch_size = 500

In [ ]:
day_1_train_losses1 = []
day_1_train_accs1 = []
day_1_valid_losses1 = []
day_1_valid_accs1 = []

day_1_train_losses2 = []
day_1_train_accs2 = []
day_1_valid_losses2 = []
day_1_valid_accs2 = []

day_1_train_losses3 = []
day_1_train_accs3 = []
day_1_valid_losses3 = []
day_1_valid_accs3 = []

In [ ]:
sess = tf.Session()

day_1_model1 = Transformer_Classifer(sess, "model1")
day_1_model2 = Transformer_Classifer(sess, "model2")
day_1_model3 = Transformer_Classifer(sess, "model3")
day_1_model1.build(batch_size, 10, 53, 4, 53, None, 4, 64, 64, tf.nn.leaky_relu, 4)
day_1_model2.build(batch_size, 10, 53, 4, 64, None, 4, 64, 64, tf.nn.leaky_relu, 4)
day_1_model3.build(batch_size, 10, 53, 6, 64, None, 6, 128, 128, tf.nn.leaky_relu, 4)

sess.run(tf.global_variables_initializer())

In [ ]:
print("Ready!")

In [ ]:
print('Learning Started!')
print("")

# train my model
for epoch in range(total_epoch):
    avg_cost1 = 0
    avg_cost2 = 0
    avg_cost3 = 0
    
    total_batch = int(len(day_1_training_lst) / batch_size)
    idx = 0
    
    if epoch == 0 :
        learning_rate = learning_rate1
    elif epoch == 40 :
        learning_rate = learning_rate2
    elif epoch == 70 :
        learning_rate = learning_rate3
    elif epoch == 80 :
        learning_rate = learning_rate4

    for i in range(total_batch):
        batch_xs, batch_ys = day_1_training_lst[idx:idx+batch_size],day_1_training_label[idx:idx+batch_size]
        
        c1, _ = day_1_model1.train(batch_xs, batch_ys, learning_rate)
        c2, _ = day_1_model2.train(batch_xs, batch_ys, learning_rate)
        c3, _ = day_1_model3.train(batch_xs, batch_ys, learning_rate)
        
        avg_cost1 += c1 / total_batch
        avg_cost2 += c2 / total_batch
        avg_cost3 += c3 / total_batch
        
        idx += batch_size
            
    #train cost & acc
    cost1, acc1 = day_1_model1.evaluate(day_1_training_lst, day_1_training_label, batch_size = batch_size)
    cost2, acc2 = day_1_model2.evaluate(day_1_training_lst, day_1_training_label, batch_size = batch_size)
    cost3, acc3 = day_1_model3.evaluate(day_1_training_lst, day_1_training_label, batch_size = batch_size)
    
    day_1_train_losses1.append(cost1)
    day_1_train_accs1.append(acc1)
    day_1_train_losses2.append(cost2)
    day_1_train_accs2.append(acc2)
    day_1_train_losses3.append(cost3)
    day_1_train_accs3.append(acc3)
    
    #valid cost & acc
    v_cost1, v_acc1 = day_1_model1.evaluate(day_1_valid_lst, day_1_valid_label, batch_size = batch_size)
    v_cost2, v_acc2 = day_1_model2.evaluate(day_1_valid_lst, day_1_valid_label, batch_size = batch_size)
    v_cost3, v_acc3 = day_1_model3.evaluate(day_1_valid_lst, day_1_valid_label, batch_size = batch_size)
    
    day_1_valid_losses1.append(v_cost1)
    day_1_valid_accs1.append(v_acc1)
    day_1_valid_losses2.append(v_cost2)
    day_1_valid_accs2.append(v_acc2)
    day_1_valid_losses3.append(v_cost3)
    day_1_valid_accs3.append(v_acc3)
    
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost1, acc1*100, v_cost1, v_acc1*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost2, acc2*100, v_cost2, v_acc2*100))
    print("epoch : ", epoch, " -- train {:.5f}({:.1f}%), valid{:.5f}({:.1f}%)".format(cost3, acc3*100, v_cost3, v_acc3*100))
    print('Accuracy:', day_1_model1.get_accuracy(day_1_valid_lst, day_1_valid_label, batch_size))
    print('Accuracy:', day_1_model2.get_accuracy(day_1_valid_lst, day_1_valid_label, batch_size))
    print('Accuracy:', day_1_model3.get_accuracy(day_1_valid_lst, day_1_valid_label, batch_size))
    
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model1.predict(day_1_training_lst, batch_size), 1), average="weighted"))
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model2.predict(day_1_training_lst, batch_size), 1), average="weighted"))
    print("train F1 score :", f1_score(np.argmax(day_1_training_label, 1), np.argmax(day_1_model3.predict(day_1_training_lst, batch_size), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model1.predict(day_1_valid_lst, batch_size), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model2.predict(day_1_valid_lst, batch_size), 1), average="weighted"))
    print("valid F1 score :", f1_score(np.argmax(day_1_valid_label, 1), np.argmax(day_1_model3.predict(day_1_valid_lst, batch_size), 1), average="weighted"))
    print(" ")

print("")
print('Learning Finished!')

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_losses"+str(idx)), label='training'+str(idx))
    plt.plot(eval("day_1_valid_losses"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_accs"+str(idx)), label='training'+str(idx))
    plt.plot(eval("day_1_valid_accs"+str(idx)), label='valid'+str(idx))
    plt.title("model"+str(idx))
    plt.grid("on")
    plt.legend()
    plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_losses"+str(idx)), label='training'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_valid_losses"+str(idx)), label='training'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_train_accs"+str(idx)), label='valid'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
for idx in range(1,4) :
    plt.plot(eval("day_1_valid_accs"+str(idx)), label='valid'+str(idx))
    
plt.grid("on")
plt.legend()
plt.show()

In [ ]:
#tf.reset_default_graph() 

<br></br><br></br><br></br>

In [ ]:
saver = tf.train.Saver()
saver.save(sess, './model_Transformer/original_user_vector_only_encoder/original_user_vector_only_encoder')

<br></br><br></br><br></br>

# Test

In [ ]:
activity = pd.read_csv("user_vector_test.csv").drop("Unnamed: 0", axis=1)

In [ ]:
activity.head()

In [ ]:
label = activity.groupby("acc_id").count().reset_index()[["acc_id"]]
label.head()

In [ ]:
id_lst = [int(x) for x in label.acc_id.tolist()]
print(len(id_lst))

In [ ]:
activity_dic = {}
for user in id_lst :
    activity_dic[user] = []

In [ ]:
print(len(activity.head().values[0]))
activity.head().values[0]

In [ ]:
for data in activity.values :
    activity_dic[int(data[0])].append([int(data[1])]+list(data[2:]))

In [ ]:
activity_lst = [list(y) for y in activity_dic.items()]
activity_lst[0]

In [ ]:
day_1_lst = [x for x in activity_lst if x[1][0][0]==1]
print(len(day_1_lst))

In [ ]:
day_1_dic = {}
day_1_id_lst = [x[0] for x in day_1_lst]

In [ ]:
START_TOKEN = [1] + [0]*52
EMPTY_TOKEN = [0,1] + [0]*51
END_TOKEN = [0,0,1] + [0]*50

In [ ]:
for user in day_1_id_lst :
    day_1_dic[(user, 0)] = START_TOKEN
    day_1_dic[(user, 1)] = EMPTY_TOKEN
    day_1_dic[(user, 2)] = EMPTY_TOKEN
    day_1_dic[(user, 3)] = EMPTY_TOKEN
    day_1_dic[(user, 4)] = EMPTY_TOKEN
    day_1_dic[(user, 5)] = EMPTY_TOKEN
    day_1_dic[(user, 6)] = EMPTY_TOKEN
    day_1_dic[(user, 7)] = EMPTY_TOKEN
    day_1_dic[(user, 8)] = EMPTY_TOKEN
    day_1_dic[(user, 9)] = END_TOKEN
    


In [ ]:
for data in day_1_lst :
    user = data[0]
    lst = data[1]
    
    for data2 in lst :
        if idx == 0 :
            first_week = data2[0]
            
            for idx2 in range(first_week) :
                day_1_dic[(user, idx2)] = START_TOKEN
                
        week = data2[0]
        day_1_dic[(user, week)] = data2[1:]

In [ ]:
day_1_total_lst = []
temp1 = list(day_1_dic.values())

last1=0

In [ ]:
for now in range(0,len(temp1)+1,9) :
    if now == 0 :
        last1 = now
        continue
    
    day_1_total_lst.append(temp1[last1:now])
    last1=now

In [ ]:
print(len(day_1_total_lst))

In [ ]:
day_1_test_lst = np.array(day_1_total_lst)

<br></br><br></br>

# Predict

In [ ]:
result_11 = np.argmax(day_1_model1.predict(day_1_test_lst), axis=1)
result_12 = np.argmax(day_1_model2.predict(day_1_test_lst), axis=1)
result_13 = np.argmax(day_1_model3.predict(day_1_test_lst), axis=1)

In [ ]:
def voting(result_lst) :
    final_result = []
    
    for data in result_lst :
        temp = sorted([(0, data.count(0)),(1, data.count(1)),(2, data.count(2)),(3, data.count(3))], key=lambda x :x[1], reverse=True)
        final_result.append(temp[0][0])

    return pd.Series(final_result)

In [ ]:
dic = {0 : "acc_id", 1:"model1", 2:"model2", 3:"model3", 4:"model4"}
result_1 = pd.DataFrame(list(zip(day_1_id_lst, result_11, result_12, result_13))).rename(columns = dic)

In [ ]:
result_1["result"] = voting(list(zip(result_1.model1,result_1.model2,result_1.model3)))

In [ ]:
result_lst = [result_1[["acc_id","result"]]]

total_result = pd.concat(result_lst).sort_values("acc_id")
total_result.head()

In [ ]:
total_result.to_csv("./result/transformer_original_user_vector_only_encoder/transformer_original_user_vector_only_encoder.csv")

In [ ]:
test_user_id_df = pd.read_csv("test_user_id.csv").drop("Unnamed: 0", axis=1)
test_user_id_df.head()

In [ ]:
test_user_id_dic = {}

for kv in test_user_id_df.values :
    test_user_id_dic[kv[1]] = kv[0]

In [ ]:
label_dic = {0 : "week", 1 : "month", 2:"2month", 3 :"retained"}

total_result["acc_id"] = total_result["acc_id"].map(lambda x: test_user_id_dic[x])
total_result["result"] = total_result["result"].map(lambda x: label_dic[x])
total_result.head()

In [ ]:
total_result2 = total_result.set_index("acc_id").rename(columns = {"result" : "label"})
total_result2.head()

In [ ]:
total_result2.to_csv("./result/transformer_original_user_vector_only_encoder/transformer_original_user_vector_only_encoder_with_acc_id.csv")